### Heroes Of Pymoli Data Analysis
* Of the 1163 active players, the vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.8%) with secondary groups falling between 15-19 (18.60%) and 25-29 (13.4%).  
-----

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [33]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

## Player Count

* Display the total number of players


In [2]:
#get a clear view of the data
purchase_data.head(3)

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88


In [3]:
#Count the number of unique screen names in the data set
number_unique_sn = len(purchase_data["SN"].value_counts())


In [4]:
#Create a data frame with that information
total_players = pd.DataFrame({"Total Players":number_unique_sn},index = [0])
total_players.head()

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [5]:
#Run Calculations
number_unique_items = len(purchase_data.loc[:,"Item ID"].unique())
average_price = round(purchase_data.loc[:,"Price"].mean(),2)
number_purchases = len(purchase_data.loc[:,"Purchase ID"].unique())
total_revenue = purchase_data.loc[:,"Price"].sum()

In [6]:
#Create summary data frame
purchase_analysis = pd.DataFrame({"Number of Unique Items":number_unique_items,
                                  "Average Price":f"${average_price}",
                                  "Number of Purchases":number_purchases,
                                  "Total Revenue":f"${total_revenue}"},index=[0])
purchase_analysis

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,$2379.77


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [7]:
#create array of gender types
genders = purchase_data["Gender"].unique()
#create array of the count of those genders (uniqe SN's only)
total_count = []
for gender in genders:
    total_count.append(len(purchase_data.loc[purchase_data["Gender"]==gender,"SN"].unique()))


In [8]:
#create data frame with the above information
gender_counts = pd.DataFrame(total_count,index=genders,columns=["Total Count"])

In [9]:
#add percentage to the above data frame
gender_counts.loc[:,"Percentage of Players"] = (gender_counts["Total Count"] / gender_counts["Total Count"].sum()) * 100
gender_counts.index.name = "Gender"
gender_counts

,Total Count,Percentage of Players
Gender,,
Male,484,84.027778
Other / Non-Disclosed,11,1.909722
Female,81,14.062500



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [16]:
#Avg Purchase Price
price = purchase_data.groupby("Gender").mean()
price = price.drop(columns=["Purchase ID","Age","Item ID"])

#Purchase Count
count = purchase_data.groupby("Gender").count()
count = count.drop(columns = ['SN', 'Age', 'Item ID', 'Item Name', 'Price'])


#Total Purchase Value
total = purchase_data.groupby("Gender").sum()
total = total.drop(columns = ['Purchase ID', 'Age', 'Item ID'])

#Create Data Frame
price_count = pd.merge(price,count,on="Gender")
total_gender_counts = pd.merge(total,gender_counts,on="Gender")
gender_stats = pd.merge(price_count,total_gender_counts,on="Gender")
gender_stats = gender_stats.rename(columns = { "Price_x" :"Average Purchase Price",
                                               "Purchase ID" : "Purchase Count",
                                               "Price_y" : "Total Purchase Value",
                                               "Total Count" : "Player Count for Gender"})

#Add Avg Total Purchase per Person Column
gender_stats.loc[:,"Avg Total Purchase per Person"] = gender_stats["Total Purchase Value"] / gender_stats["Player Count for Gender"]
gender_stats = gender_stats.drop(columns = ["Percentage of Players"])
gender_stats

,Average Purchase Price,Purchase Count,Age Range,Total Purchase Value,Player Count for Gender,Avg Total Purchase per Person
Gender,,,,,,
Female,3.203009,113,113,361.94,81,4.468395
Male,3.017853,652,652,1967.64,484,4.065372
Other / Non-Disclosed,3.346000,15,15,50.19,11,4.562727


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [11]:
age_data = purchase_data

age_bins = np.array([0,10,15,20,25,30,35,40,100])
bin_names = ["<10","10-14","15-19","20-24",
             "25-29","30-34","35-49","40+"]

age_data["Age Range"] = pd.cut(purchase_data["Age"],age_bins,labels = bin_names, right=False)


age_demo = age_data.groupby("Age Range")["SN"].nunique().to_frame()
age_demo = age_demo.rename(columns = {"SN":"Total Count"})
age_demo["Percentage of Players"] = (age_demo["Total Count"] / age_demo["Total Count"].sum()) * 100
age_demo

,Total Count,Percentage of Players
Age Range,,
<10,17,2.951389
10-14,22,3.819444
15-19,107,18.576389
20-24,258,44.791667
25-29,77,13.368056
30-34,52,9.027778
35-49,31,5.381944
40+,12,2.083333


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [27]:
#Use the arrays defined above
age_purchase_analysis = purchase_data
age_purchase_analysis["Age Range"] = pd.cut(purchase_data["Age"],age_bins,labels = bin_names, right=False)

#Purchase Count
pc = age_purchase_analysis.groupby("Age Range")["Purchase ID"].count().to_frame()
pc = pc.rename(columns = {"Purchase ID":"Purchase Count"})
#Avgerage Purchase Price
app = age_purchase_analysis.groupby("Age Range")["Price"].mean().to_frame()
app = app.rename(columns = {"Price":"Average Purchase Price"})

#Total Purchase Value
tpv = age_purchase_analysis.groupby("Age Range")["Price"].sum().to_frame()
tpv = tpv.rename(columns = {"Price":"Total Purchase Value"})

#Total Players
tp = age_purchase_analysis.groupby("Age Range")["SN"].nunique().to_frame()
tp = tp.rename(columns = {"SN":"Total Players for Age Range"})
tp

#Merge
first_merge = pd.merge(pc,app, on="Age Range")
second_merge = pd.merge(tpv,tp, on="Age Range")
age_purchase_demo = pd.merge(first_merge,second_merge,on="Age Range")

#Add Avgerage Purchase Total Per Person
age_purchase_demo["Average Purchase Total Per Person"] = age_purchase_demo["Total Purchase Value"] / age_purchase_demo["Total Players for Age Range"]

age_purchase_demo

,Purchase Count,Average Purchase Price,Total Purchase Value,Total Players for Age Range,Average Purchase Total Per Person
Age Range,,,,,
<10,23,3.353478,77.13,17,4.537059
10-14,28,2.956429,82.78,22,3.762727
15-19,136,3.035956,412.89,107,3.858785
20-24,365,3.052219,1114.06,258,4.318062
25-29,101,2.900990,293.00,77,3.805195
30-34,73,2.931507,214.00,52,4.115385
35-49,41,3.601707,147.67,31,4.763548
40+,13,2.941538,38.24,12,3.186667


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [41]:
top_spenders = purchase_data.groupby("SN")["Price"].agg(["count","mean","sum"])
top_spenders = top_spenders.rename(columns = {"count":"Purchase Count",
                                              "mean":"Average Purchase Price",
                                              "sum":"Total Purchase Value"})


sorted_top_spenders = top_spenders.sort_values("Total Purchase Value",ascending=False)

sorted_top_spenders.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,3.792000,18.96
Idastidru52,4,3.862500,15.45
Chamjask73,3,4.610000,13.83
Iral74,4,3.405000,13.62
Iskadarya95,3,4.366667,13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [54]:
#The items price should be the same for all instances so running mean will return the price
item_profit = purchase_data.groupby(["Item ID","Item Name"])["Price"].agg(["count","mean"])
item_profit = item_profit.rename(columns = {"count":"Purchase Count","mean":"Item Price"})

#Create Column for total Purchase Value
item_profit["Total Purchase Value"] = item_profit["Purchase Count"] * item_profit["Item Price"]

#Sort based on item that has returned the most
popular_items = item_profit.sort_values("Purchase Count",ascending=False)

popular_items.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,50.76
145,Fiery Glass Crusader,9,4.58,41.22
108,"Extraction, Quickblade Of Trembling Hands",9,3.53,31.77
82,Nirvana,9,4.90,44.10
19,"Pursuit, Cudgel of Necromancy",8,1.02,8.16


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [53]:
#Sort based on item that has returned the most
high_gross_items = item_profit.sort_values("Total Purchase Value",ascending=False)

high_gross_items.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,50.76
82,Nirvana,9,4.90,44.10
145,Fiery Glass Crusader,9,4.58,41.22
92,Final Critic,8,4.88,39.04
103,Singed Scalpel,8,4.35,34.80
